<a href="https://colab.research.google.com/github/zenon10/POC-OCR/blob/main/extract_text_vSB20230209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Installations**

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
## For pdf
! pip install pypdfium2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!sudo apt install tesseract-ocr
!pip install pytesseract

#Explicitly for french language
! sudo apt-get install tesseract-ocr-fra

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-fra is already the newest version (1:4.00~git30-7274cfa-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.


In [3]:
## For textract 
!apt-get install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils \
     pstotext tesseract-ocr \
     flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig libasound2-dev libpulse-dev
!pip install git+https://github.com/deanmalmgren/textract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python-dev-is-python2' instead of 'python-dev'
libjpeg-dev is already the newest version (8c-2ubuntu8).
antiword is already the newest version (0.37-16).
lame is already the newest version (3.100-3).
libmad0 is already the newest version (0.15.1b-10ubuntu1).
libsox-fmt-mp3 is already the newest version (14.4.2+git20190427-2).
pstotext is already the newest version (1.9-6build1).
python-dev-is-python2 is already the newest version (2.7.17-4).
sox is already the newest version (14.4.2+git20190427-2).
swig is already the newest version (4.0.1-5build1).
tesseract-ocr is already the newest version (4.1.1-2build2).
unrtf is already the newest version (0.21.10-clean-1).
libasound2-dev is already the newest version (1.2.2-2.1ubuntu2.5).
libpulse-dev is already the newest version (1:13.99.1-1ubuntu3.13).
libxml2-dev is already the newest version (2.9.10+dfsg-5ubuntu0.20.04.5).
libxsl

##**Libraries**

In [4]:
# Library to work with Operating Syatem
import os

# Library to create PDF, inspection, manupulation and rendering(turning into images)
import pypdfium2 as pdfium

# Regex lirary to manipulate string
import re

from PIL import Image

import textract

In [5]:
import os
import glob
import pandas as pd
import pytesseract
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image

##**Defining Function**

In [6]:
def pdf_to_img(input_file_path, output_folder_path):
    
    input_file_name = os.path.split(input_file_path)[-1]
    print('Extracting text from {}'.format(input_file_name))
    
    # PdfDocument (supports file path strings, bytes, and byte buffers)
    pdf_info = pdfium.PdfDocument(input_file_path)
    
    # get the number of pages in the document
    n_pages = len(pdf_info)
    print('Number of pages {}'.format(n_pages))
    
    page_indices = [i for i in range(n_pages)]    
    renderer = pdf_info.render_to(pdfium.BitmapConv.pil_image, page_indices = page_indices, scale = 300/72)

    page_number = 1
    
    print("Saving images...")
    for images in renderer:
      images.save(output_folder_path + "/" + str(page_number)  + ".jpeg")
      page_number +=1

In [7]:
def img_to_txt(input_img_path, output_txt_path):
    
    try:
        path = sorted(glob.glob(input_img_path + "/*.jpeg"))
        print("Images present in input image folder", [i for i in path])
    except:
        raise ValueError('Image not in jpeg')
    
        
    ## Blank string to concatenate all text string outputs from ocr for each image
    combined_text = ''

    l = len(path)

    for i in range(l):
      i = i + 1

      img = input_img_path + '/' + str(i) + '.jpeg'
      ## Extracting text from image
      text = pytesseract.image_to_string(Image.open(img), lang='fra')
      
      ## Concatenating text-string outputs from each image into one master text-string
      combined_text = ("\n**" + str(i) + " **\n").join([combined_text, text])
        
    
    ## Open a file with access mode 'w'
    file_object2 = open(output_txt_path, 'w', encoding="utf8")
    file_object2.write(combined_text)
    file_object2.close()

In [8]:
def extract_text(input_file_path, output_file_path, extra_dir=None):
    '''
    This function takes French PDFs, doc, docx, excels, 
    extract text and save it in a txt file.

    Input parameters
        input_file_path: Input file location
        output_file_path: Output file location
        extra_dir: Directory to save additional output files for PDFs
    Output
        For PDF files, the extra_dir will have jpeg files and output_file_path will have txt files
        For doc, docx and the excel, output_file_path will be only txt files
    '''
    
    if os.path.exists(input_file_path):
      print("Text extraction process starting for...",input_file_path)
      
      ## Storing extension of input_file_path
      file_name_extn = os.path.basename(input_file_path).split('.')[1]
      
      ## Keeping a list of acceptable extensions
      pdf_extension = ['PDF']
      non_pdf_extension = ['DOC', 'DOCX', 'XLSX']

      ## Applying text extraction process based on formats
      if file_name_extn.upper() in pdf_extension:
        
        ## Converting each page of PDF to an image and saving image to extra_dir
        pdf_to_img(input_file_path, extra_dir)

        ## Extracting text from each image and storing combined text in output_file_path
        img_to_txt(extra_dir, output_file_path)

      elif file_name_extn.upper() in non_pdf_extension:
        text_file = textract.process(input_file_path)
        
        ## Saving text in output_file_path
        with open(output_file_path, 'w') as f:
                    f.write(text_file.decode('utf8', 'strict'))
      
      else:
        raise ValueError("File does not match required formats - PDF, doc, docx, xlsx")
    else:
      raise ValueError("File does not exist!!")

##**Testing**

In [10]:
input_file_path = '/content/drive/MyDrive/f1.pdf'
output_file_path = '/content/drive/MyDrive/extract_text_extra_dir_checking/f1.txt'
extra_dir= '/content/drive/MyDrive/extract_text_extra_dir_checking'
extract_text(input_file_path, output_file_path, extra_dir)

Text extraction process starting for... /content/drive/MyDrive/f1.pdf
Extracting text from f1.pdf
Number of pages 6
Saving images...
Images present in input image folder ['/content/drive/MyDrive/extract_text_extra_dir_checking/1.jpeg', '/content/drive/MyDrive/extract_text_extra_dir_checking/2.jpeg', '/content/drive/MyDrive/extract_text_extra_dir_checking/3.jpeg', '/content/drive/MyDrive/extract_text_extra_dir_checking/4.jpeg', '/content/drive/MyDrive/extract_text_extra_dir_checking/5.jpeg', '/content/drive/MyDrive/extract_text_extra_dir_checking/6.jpeg']


Calling .py file

In [15]:
!python /content/drive/MyDrive/pyCheck/copy_of_extract_text_v20230207_vf.py

Installing pypdfium2...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Installation done...
Installing pytesseract...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Installation done...
Text extraction process starting for... /content/drive/MyDrive/f1.pdf
Extracting text from f1.pdf
Number of pages 6
Saving images...
Images present in input image folder ['/content/drive/MyDrive/extract_text_extra_dir_checking/1.jpeg', '/content/drive/MyDrive/extract_text_extra_dir_checking/2.jpeg', '/content/drive/MyDrive/extract_text_extra_dir_checking/3.jpeg', '/content/drive/MyDrive/extract_text_extra_dir_checking/4.jpeg', '/content/drive/MyDrive/extract_text_extra_dir_checking/5.jpeg', '/content/drive/MyDrive/extract_text_extra_dir_checking/6.jpeg']
